__Vi bygger om logiken för att plocka ut 14 hästar när vi plockat bort de hästar som är i VNUM (1,2,3).__

In [28]:
# Common imports
import numpy as np
import os
import pandas as pd

In [29]:
# Necessary Sklearn objects used in the analysis
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn import preprocessing

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

In [30]:
# DataPiplene är en pythonfil som innehåller den egenkostruerade klassen DataFrameSelector
from DataPipeline import DataFrameSelector
from sklearn.externals import joblib

__Läser in filen som ska köras__

In [31]:
df1 = pd.read_excel('TestDataV75TillUffeF20200307.xlsx')

In [32]:
df1 = df1[df1.VNUM.isin([1,2,3])]    

In [33]:
df1 = df1.sort_values(['Lopp','Hast'], ascending = ['True','False']).reset_index()

In [34]:
# Importera model som ska användas
rf = joblib.load('Travmodel_top1.pkl')

In [35]:
# Import av datapipeline som ska användas
full_pipeline = joblib.load('Pipeline_top1.pkl')

__Kör modellen och plockar listan med högsta score__

In [36]:
features = full_pipeline.fit_transform(df1)

In [37]:
predict = rf.predict_proba(features)

In [38]:
# Gör om scorad array till en lista - multidimensionell
stack = predict.tolist()

In [39]:
# Sedan plockar vi den scorade sannolikheten att vara vinnare - 1
# Transformerar denna sannolikhet till en dataframe
last = []
for x in stack:
    last.append(x[1])
scored = {'Score':last}
# Konverterar till Dataframe
df_scored = pd.DataFrame.from_dict(scored)

In [40]:
df_predict2 = df1.merge(df_scored,  right_index = True, left_index = True) 

In [42]:
df1_out = df_predict2[['Lopp','Hast','Score','E_P_Num']].sort_values(['Score'], ascending = [False])
df1_out

,Lopp,Hast,Score,E_P_Num
4,2,8,0.390974,1
7,3,2,0.355432,2
1,1,5,0.325466,6
8,3,5,0.294173,2
13,5,4,0.286228,3
19,7,9,0.271612,5
2,1,8,0.268476,6
9,4,1,0.242839,4
18,7,5,0.223406,5
10,4,6,0.216921,4


__Export till excel__

In [15]:
df1_out.to_excel('TestScoredTop21_0307.xlsx', index = False)